In [5]:
import pandas as pd
import json
import os

In [6]:
BASE_PATH = "../data"
TRAIN_PATH = os.path.join(BASE_PATH, "train")
EVAL_PATH = os.path.join(BASE_PATH, "eval")
DUMMY_PATH = os.path.join(BASE_PATH, "dummy")

In [7]:
train_ratings_path = os.path.join(TRAIN_PATH, 'train_ratings.csv')
train_ratings = pd.read_csv(train_ratings_path)
users = train_ratings['user'].unique()

users = users[:10]

dummys = []
for user in users:
    dummys.append(train_ratings[train_ratings['user'] == user][:100])
dummy_r = pd.concat(dummys)

dummy_train_ratings_path = os.path.join(DUMMY_PATH, 'train_ratings.csv')
dummy_r.to_csv(dummy_train_ratings_path, index=False)

items = dummy_r['item'].unique()

In [8]:
sub_path = os.path.join(EVAL_PATH, 'sample_submission.csv')
sub = pd.read_csv(sub_path)

dummys = []
for user in users:
    r_i = dummy_r[dummy_r['user'] == user]['item']
    
    df = sub[sub['user'] == user][:10]
    tmp_items = list(set(items) - set(r_i))
    tmp_items = tmp_items[:len(df['item'])]
    df['item'] = tmp_items
    dummys.append(df)
dummy_s = pd.concat(dummys)

dummy_s_path = os.path.join(DUMMY_PATH, 'sample_submission.csv')
dummy_s.to_csv(dummy_s_path, index=False)

In [9]:
def create_attr_dummy(file_name):
    attr_path = os.path.join(TRAIN_PATH, file_name)
    with open(attr_path) as f:
        item2attrs = json.load(f)
    
    new_item2attrs = {}
    for item_id, attrs in item2attrs.items():
        if int(item_id) in items:
            new_item2attrs[item_id] = attrs

    dummy_attr_path = os.path.join(DUMMY_PATH, file_name)
    with open(dummy_attr_path, 'w') as f:    
        json.dump(new_item2attrs, f)
    

In [10]:
for file_name in os.listdir(TRAIN_PATH):
    if not file_name.endswith('.json'):
        continue
    
    create_attr_dummy(file_name)